In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from flexml.regression import Regression
import pandas as pd

In [3]:
df = pd.read_csv("../tests/test_data/wine_regression.csv")
df.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alcohol               178 non-null    float64
 1   Malic_Acid            178 non-null    float64
 2   Ash                   178 non-null    float64
 3   Ash_Alcanity          178 non-null    float64
 4   Magnesium             178 non-null    int64  
 5   Total_Phenols         178 non-null    float64
 6   Flavanoids            178 non-null    float64
 7   Nonflavanoid_Phenols  178 non-null    float64
 8   Proanthocyanins       178 non-null    float64
 9   Color_Intensity       178 non-null    float64
 10  Hue                   178 non-null    float64
 11  OD280                 178 non-null    float64
 12  Proline               178 non-null    int64  
 13  Customer_Segment      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [5]:
reg_exp = Regression(df, target_col = "Customer_Segment")
reg_exp.start_experiment()

INFO | 2024-06-24 23:50:52 | flexml.structures.supervised_base.start_experiment | [PROCESS] Training the ML models
100%|██████████| 8/8 [00:02<00:00,  3.24it/s]
INFO | 2024-06-24 23:50:55 | flexml.structures.supervised_base.start_experiment | [PROCESS] Model training is finished!


              model_name      r2     mae     mse    rmse
0      CatBoostRegressor  0.9544  0.1034  0.0272  0.1649
1          LGBMRegressor  0.9160  0.1469  0.0500  0.2236
2       LinearRegression  0.8871  0.2046  0.0672  0.2592
3                  Ridge  0.8871  0.2040  0.0672  0.2592
4           XGBRegressor  0.7388  0.1238  0.1555  0.3943
5  DecisionTreeRegressor  0.7015  0.1333  0.1778  0.4217
6             ElasticNet  0.5858  0.4087  0.2467  0.4967
7                  Lasso  0.4330  0.4892  0.3377  0.5811


In [8]:
reg_exp.show_model_stats()

              model_name      r2     mae     mse    rmse
0      CatBoostRegressor  0.9544  0.1034  0.0272  0.1649
1          LGBMRegressor  0.9160  0.1469  0.0500  0.2236
2       LinearRegression  0.8871  0.2046  0.0672  0.2592
3                  Ridge  0.8871  0.2040  0.0672  0.2592
4           XGBRegressor  0.7388  0.1238  0.1555  0.3943
5  DecisionTreeRegressor  0.7015  0.1333  0.1778  0.4217
6             ElasticNet  0.5858  0.4087  0.2467  0.4967
7                  Lasso  0.4330  0.4892  0.3377  0.5811


In [6]:
best_model = reg_exp.get_best_models()
print(best_model)

In [7]:
best_model_2 = reg_exp.get_best_models(top_n_models=2)[0]
print(best_model_2)

LGBMRegressor(verbose=-1)


In [7]:
reg_exp.tune_model(best_model, tuning_method="optuna", n_trials = 4, cv = 2)

INFO | 2024-06-24 23:51:01 | flexml.structures.supervised_base.tune_model | [PROCESS] Model Tuning process is started
[I 2024-06-24 23:51:01,571] A new study created in memory with name: no-name-ac32a05c-7c38-440e-9edf-7f9c902e7744


0:	learn: 0.7195677	total: 17.8ms	remaining: 3.69s
0:	learn: 0.5870762	total: 16.8ms	remaining: 4.56s
1:	learn: 0.6767507	total: 29ms	remaining: 2.99s
1:	learn: 0.4573839	total: 26.4ms	remaining: 3.56s
2:	learn: 0.6311449	total: 50.8ms	remaining: 3.47s
2:	learn: 0.3684524	total: 47.8ms	remaining: 4.29s
0:	learn: 0.7023194	total: 4.37ms	remaining: 1.02s
0:	learn: 0.6479542	total: 2.18ms	remaining: 322ms
3:	learn: 0.2871533	total: 53.6ms	remaining: 3.59s
3:	learn: 0.5848227	total: 63.4ms	remaining: 3.23s
1:	learn: 0.5586391	total: 4.36ms	remaining: 321ms
1:	learn: 0.6548608	total: 7.86ms	remaining: 912ms
2:	learn: 0.4591263	total: 6.45ms	remaining: 314ms
2:	learn: 0.5984372	total: 11.3ms	remaining: 869ms
3:	learn: 0.3863493	total: 8.56ms	remaining: 310ms
4:	learn: 0.2307403	total: 60ms	remaining: 3.21s
4:	learn: 0.3374136	total: 11.7ms	remaining: 338ms
5:	learn: 0.2877434	total: 14.7ms	remaining: 351ms
3:	learn: 0.5485324	total: 19.4ms	remaining: 1.11s
6:	learn: 0.2537580	total: 17.2ms	r

[I 2024-06-24 23:51:02,450] Trial 3 finished with value: 0.9604866588174661 and parameters: {'allow_writing_files': False, 'n_estimators': 149, 'learning_rate': 0.24732840690333527, 'depth': 5, 'l2_leaf_reg': 5, 'border_count': 122}. Best is trial 3 with value: 0.9604866588174661.


120:	learn: 0.0143888	total: 799ms	remaining: 746ms
121:	learn: 0.0141254	total: 803ms	remaining: 737ms
122:	learn: 0.0139296	total: 809ms	remaining: 730ms
123:	learn: 0.0138611	total: 816ms	remaining: 724ms
124:	learn: 0.0138119	total: 821ms	remaining: 716ms
125:	learn: 0.0136018	total: 825ms	remaining: 708ms
126:	learn: 0.0133078	total: 832ms	remaining: 701ms
127:	learn: 0.0129666	total: 839ms	remaining: 695ms
51:	learn: 0.0528527	total: 905ms	remaining: 2.71s
70:	learn: 0.0017835	total: 898ms	remaining: 2.54s
52:	learn: 0.0512460	total: 924ms	remaining: 2.7s
71:	learn: 0.0017401	total: 910ms	remaining: 2.53s
72:	learn: 0.0016834	total: 925ms	remaining: 2.52s
128:	learn: 0.0128844	total: 883ms	remaining: 719ms
73:	learn: 0.0016143	total: 931ms	remaining: 2.49s
129:	learn: 0.0126841	total: 894ms	remaining: 715ms
74:	learn: 0.0015136	total: 941ms	remaining: 2.47s
130:	learn: 0.0123927	total: 898ms	remaining: 706ms
53:	learn: 0.0497301	total: 960ms	remaining: 2.74s
131:	learn: 0.0120358

[I 2024-06-24 23:51:03,068] Trial 2 finished with value: 0.9611029553899 and parameters: {'allow_writing_files': False, 'n_estimators': 234, 'learning_rate': 0.11813369773896709, 'depth': 6, 'l2_leaf_reg': 2, 'border_count': 124}. Best is trial 2 with value: 0.9611029553899.


146:	learn: 0.0000214	total: 1.45s	remaining: 1.23s
103:	learn: 0.0128494	total: 1.46s	remaining: 1.46s
147:	learn: 0.0000201	total: 1.45s	remaining: 1.21s
104:	learn: 0.0124148	total: 1.47s	remaining: 1.44s
148:	learn: 0.0000188	total: 1.45s	remaining: 1.2s
105:	learn: 0.0121477	total: 1.48s	remaining: 1.42s
149:	learn: 0.0000182	total: 1.47s	remaining: 1.19s
106:	learn: 0.0118998	total: 1.5s	remaining: 1.41s
150:	learn: 0.0000166	total: 1.48s	remaining: 1.19s
107:	learn: 0.0116500	total: 1.5s	remaining: 1.39s
151:	learn: 0.0000156	total: 1.49s	remaining: 1.18s
152:	learn: 0.0000150	total: 1.5s	remaining: 1.16s
108:	learn: 0.0113394	total: 1.52s	remaining: 1.38s
153:	learn: 0.0000138	total: 1.5s	remaining: 1.15s
154:	learn: 0.0000131	total: 1.51s	remaining: 1.14s
109:	learn: 0.0110977	total: 1.53s	remaining: 1.36s
155:	learn: 0.0000121	total: 1.52s	remaining: 1.13s
110:	learn: 0.0108805	total: 1.53s	remaining: 1.34s
156:	learn: 0.0000117	total: 1.52s	remaining: 1.12s
111:	learn: 0.010

[I 2024-06-24 23:51:03,790] Trial 0 finished with value: 0.9215476520044927 and parameters: {'allow_writing_files': False, 'n_estimators': 208, 'learning_rate': 0.11455662803824733, 'depth': 9, 'l2_leaf_reg': 3, 'border_count': 34}. Best is trial 2 with value: 0.9611029553899.
[I 2024-06-24 23:51:03,801] Trial 1 finished with value: 0.9101786300043994 and parameters: {'allow_writing_files': False, 'n_estimators': 272, 'learning_rate': 0.28849380682382986, 'depth': 7, 'l2_leaf_reg': 1, 'border_count': 107}. Best is trial 2 with value: 0.9611029553899.
INFO | 2024-06-24 23:51:03 | flexml.structures.supervised_base.tune_model | [PROCESS] Model Tuning process is finished


197:	learn: 0.0017897	total: 2.06s	remaining: 104ms
248:	learn: 0.0000001	total: 2.04s	remaining: 189ms
198:	learn: 0.0017584	total: 2.06s	remaining: 93.3ms
249:	learn: 0.0000001	total: 2.05s	remaining: 181ms
199:	learn: 0.0017221	total: 2.07s	remaining: 82.7ms
250:	learn: 0.0000001	total: 2.06s	remaining: 172ms
200:	learn: 0.0016849	total: 2.07s	remaining: 72.2ms
251:	learn: 0.0000001	total: 2.06s	remaining: 163ms
201:	learn: 0.0016491	total: 2.08s	remaining: 61.7ms
252:	learn: 0.0000001	total: 2.07s	remaining: 155ms
202:	learn: 0.0015986	total: 2.08s	remaining: 51.4ms
253:	learn: 0.0000001	total: 2.07s	remaining: 147ms
203:	learn: 0.0015753	total: 2.09s	remaining: 41ms
254:	learn: 0.0000001	total: 2.08s	remaining: 138ms
255:	learn: 0.0000001	total: 2.08s	remaining: 130ms
204:	learn: 0.0015377	total: 2.1s	remaining: 30.7ms
256:	learn: 0.0000001	total: 2.09s	remaining: 122ms
205:	learn: 0.0015012	total: 2.1s	remaining: 20.4ms
257:	learn: 0.0000001	total: 2.09s	remaining: 114ms
206:	lea

In [9]:
reg_exp.tuned_model

LGBMRegressor(colsample_bytree=0.6897342503498265,
              learning_rate=0.14060791573489057, max_depth=7, n_estimators=141,
              reg_alpha=0, reg_lambda=0, subsample=0.5588471072008101,
              verbose=-1)

In [10]:
reg_exp.tuned_model_score

0.9223121650118092